# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Environment, ScriptRunConfig
Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

{
    "assetId": null,
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230509.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "sklearn-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
     

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [16]:
ws = Workspace.from_config()
experiment_name = 'heart_failure_prediction_hyperml4'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


Workspace name: quick-starts-ws-250923
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-250923


In [17]:

param_sampling = RandomParameterSampling(
    { 
        '--C': choice(0.1, 1.0,10),
        '--max_iter': choice(['5', '10', '20'])
    }
)

early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=0)
#policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)



# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
cluster_name = "Udacity-Capstone-Project-Cluster"
src = ScriptRunConfig(source_directory='./',
                                script='train.py',
                                environment=sklearn_env, 
                                compute_target=cluster_name)


hyperdrive_config  = HyperDriveConfig(hyperparameter_sampling = param_sampling , primary_metric_name = "Accuracy", primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, max_total_runs=20, max_concurrent_runs=2, max_duration_minutes=30, policy=early_termination_policy, estimator=None, run_config=src, resume_from=None, resume_child_runs=None, pipeline=None, debug_flag=None, custom_run_id=None)

hdr= experiment.submit(config=hyperdrive_config)



## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [18]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [17]:
import joblib
from azureml.train.hyperdrive.run import HyperDriveRun
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = HyperDriveRun.get_best_run_by_primary_metric(hdr)
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']#['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
#print('\n learning rate:',parameter_values[3])
#print('\n keep probability:',parameter_values[5])
#print('\n batch size:',parameter_values[7])

best_run.get_file_names()

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

